In [9]:
# libraries
import os
import joblib
from osgeo import gdal
import pandas as pd
import numpy as np
import xarray as xr
import glob
import re
import rioxarray
import dask.array
from dask.distributed import Client, progress

In [2]:
client = Client(n_workers=15, threads_per_worker=2)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 15
Total threads: 30,Total memory: 240.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43789,Workers: 15
Dashboard: /proxy/8787/status,Total threads: 30
Started: Just now,Total memory: 240.00 GiB
Comm: tcp://127.0.0.1:38097,Total threads: 2
Dashboard: /proxy/37857/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:37225,


In [4]:
workingPath = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/"

In [10]:
# function for getting directory of input data
def get_directories_with_number_and_ending(directory_path, ending):
    directories = []
    if ending:
        pattern = re.compile(r'^\d+.*{}$'.format(re.escape(ending)))  # Match directory names with number at the beginning and specific ending
    else:
        pattern = re.compile(r'^\d+$')  # Match directory names with only numbers

    for entry in os.scandir(directory_path):
        if entry.is_dir():
            directory_name = entry.name
            if pattern.match(directory_name):
                directories.append(directory_name)

    return directories

In [11]:
### define the path of input data, set the reference file for spatial resample
inputData = workingPath+"1input_data/"
# get all the filefolders named as year
year_list = get_directories_with_number_and_ending(workingPath+"1input_data/", "global")
# the input data from year[0] to year[..], based on the process id in sbatch script
year = year_list[4-1]
print(year)

2015global


In [12]:
import geopandas as gpd
from shapely.geometry import box

shapefile_path = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/Emulator/input_data/EuropeBoundary.shp"
gdf = gpd.read_file(shapefile_path)
bbox = gdf.total_bounds
bbox

array([-31.28903052,  34.93055094,  68.93136141,  81.85192337])

In [13]:
### 0) read era5land data
# select Europe data of ERA5Land and save it as netcdf
all1 = xr.open_mfdataset(inputData+year+"/era5land/era5-land*.nc", chunks={"time":-1,"latitude":400,"longitude":400})
lon_name = 'longitude'
all1['longitude_adjusted'] = xr.where(
    all1[lon_name] > 180,
    all1[lon_name] - 360,
    all1[lon_name])
all1 = (
    all1
    .swap_dims({lon_name: 'longitude_adjusted'})
    .sel(**{'longitude_adjusted': sorted(all1.longitude_adjusted)})
    .drop(lon_name))
all1 = all1.rename({'longitude_adjusted': lon_name})
all1 = all1.sel(
    latitude=slice(bbox[3], bbox[1]),  
    longitude=slice(bbox[0], bbox[2])  
)

In [14]:
all1

<xarray.Dataset>
Dimensions:    (latitude: 469, time: 8760, longitude: 1002)
Coordinates:
  * latitude   (latitude) float32 81.8 81.7 81.6 81.5 ... 35.3 35.2 35.1 35.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-12-31T23:00:00
  * longitude  (longitude) float32 -31.2 -31.1 -31.0 -30.9 ... 68.7 68.8 68.9
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(744, 318, 312), meta=np.ndarray>
    sp         (time, latitude, longitude) float32 dask.array<chunksize=(744, 318, 312), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(744, 318, 312), meta=np.ndarray>
    strd       (time, latitude, longitude) float32 dask.array<chunksize=(744, 318, 312), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(744, 318, 312), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(744, 318, 312), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(744, 318, 312), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(744, 318, 312), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-19 23:48:36 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [15]:
all1.chunks

Frozen({'time': (744, 672, 744, 720, 744, 720, 744, 744, 720, 744, 720, 744), 'latitude': (318, 151), 'longitude': (312, 400, 290)})

In [ ]:
write_job = all1.to_netcdf('/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2015global/era5land/era5land2015_10km2.nc',format='NETCDF4')
with ProgressBar():
    print(f"Writing to {out_file}")
    write_job.compute()

# test is the exported netcdf complete

In [ ]:
lat1 = 40
lat2 = 60
lon1 = 2
lon2 = 22

In [ ]:
test = xr.open_dataset(inputData+year+"/era5land/era5land2015_10km1.nc", chunks={"latitude":51, "longitude":51}) 
test = test.sel(
    latitude=slice(lat2,lat1),  
    longitude=slice(lon1, lon2)  
)

In [ ]:
a = np.nanmean(test['ssrd'], axis=(1,2))
nan_mask = np.isnan(a)

# 使用 np.sum() 来统计True的数量，即NaN值的数量
nan_count = np.sum(nan_mask)

In [ ]:
nan_count